# GAM Coach: Binary Classification

In this notebook, we will learn how to generate *diverse* and *customizable* counterfactual explanations for Generalized Additive Models (GAMs).

## 1. Install GAM Coach

First, we need to install GAM Coach through pip.

In [1]:
%%capture
!pip install gamcoach

## 2. Train an EBM Model

Let's train a binary classification model using EBM on the Lending Club dataset. The dataset contains financial information (18 features) of 5000 loan applicants. The target variable has a binary value: `1` if the person paid off the loan and `0` if the person failed to pay off the loan. To learn more about how we generate this dataset, check out our [GAM Coach paper]().

We can use this dataset to train a binary classifier to make future loan decisions (only for demo purpose).

In [2]:
import gamcoach as coach
import numpy as np
import pandas as pd

import urllib.request
import json

from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter

SEED = 101221

In [3]:
# Load a pre-processed lending club dataset
ca_data_url = 'https://gist.githubusercontent.com/xiaohk/06266553d43e591817914bfe52ec9b60/raw/c190b7cea837739797336d690fe44df9d8f9384c/lending-club-data-5000-ca.json'

with urllib.request.urlopen(ca_data_url) as url:
    data = json.loads(url.read().decode())

# Load the training data
x_all = np.array(data['x_all'])
y_all = np.array(data['y_all']) # `y_all`: 1 if paid off, 0 if failed to pay off

# Load some meta data
feature_names = data['feature_names']
# feature_types = data['feature_types']
feature_types = ['continuous' if t == 'continuous' else 'nominal' for t in data['feature_types']]

cont_index = data['cont_index']
cat_index = data['cat_index']

# Use float to encode continuous features and string for categorical features
for i, t in enumerate(feature_types):
    if t == 'continuous':
        x_all[:, i] = x_all[:, i].astype(float)
    elif t == 'categorical':
        x_all[:, i] = x_all[:, i].astype(str)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [4]:
# Create a dataFrame to validate that we load the correct data
df = pd.DataFrame(x_all)
df.columns = feature_names

df.head()

NameError: name 'x_all' is not defined

In [4]:
# Train a binary classifier
x_train, x_test, y_train, y_test = train_test_split(x_all,
                                                    y_all,
                                                    test_size=0.2,
                                                    random_state=SEED)

# Use sample weight to combat class imbalance
weight = np.bincount(y_train)[1] / np.bincount(y_train)[0]
x_train_sample_weights = [weight if y_train[i] == 0 else 1 for i in range(len(y_train))]

ebm = ExplainableBoostingClassifier(feature_names,
                                    feature_types,
                                    random_state=SEED)

ebm.fit(x_train, y_train, sample_weight=x_train_sample_weights)

ExplainableBoostingClassifier(feature_names=['loan_amnt', 'term', 'emp_length',
                                             'home_ownership', 'annual_inc',
                                             'verification_status', 'purpose',
                                             'dti', 'earliest_cr_line',
                                             'open_acc', 'pub_rec', 'revol_bal',
                                             'revol_util', 'total_acc',
                                             'application_type', 'mort_acc',
                                             'pub_rec_bankruptcies',
                                             'fico_score'],
                              feature_types=['continuous', 'nominal', 'nominal',
                                             'nominal', 'continuous', 'nominal',
                                             'nominal', 'continuous',
                                             'continuous', 'continuous',
                                             'nominal', 'continuous',
                                             'continuous', 'continuous',
                                             'nominal', 'nominal', 'nominal',
                                             'continuous'],
                              random_state=101221)

In [5]:
# Evaluate our model

y_pred = ebm.predict(x_test)
y_pred_prob = ebm.predict_proba(x_test)[:, 1]

print(Counter(y_pred))
print()

accuracy = metrics.accuracy_score(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred_prob)
f1 = metrics.f1_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

tn = confusion_matrix[0, 0]
fn = confusion_matrix[1, 0]
fp = confusion_matrix[0, 1]
tp = confusion_matrix[1, 1]
specificity = tn / (tn + fp)

temp = ('accuracy: {:.4f} \nauc: {:.4f} \nrecall:{:.4f} \nprecision: {:.4f} '+
    'specificity: {:.4f} \nf1: {:.4f}\nbalanced accuracy: {:.4f}')
print(temp.format(accuracy, auc, recall, precision, specificity, f1, balanced_accuracy))
print()

print('confusion matrix:\n', confusion_matrix)

Counter({1: 637, 0: 363})

accuracy: 0.6630 
auc: 0.6775 
recall:0.6899 
precision: 0.8556 specificity: 0.5619 
f1: 0.7638
balanced accuracy: 0.6259

confusion matrix:
 [[118  92]
 [245 545]]


## 3. Generate Counterfactual Explanations

We have trained an EBM model with quite decent performance on predicting if a loan borrower can successfully pay off the loan.

Suppose we use this model to decide if we should approve a loan application, we need to think about model interpretability. EBM models are glass-box models, where we know how exactly these models make predictions. If we want to explain loan refusal to a layperson, it is more helpful to provide **counterfactual explanations**. These explanation could say *''if your annual income were 5k higher, your application would be approved.''* They not only inform end users with key features that contribute to the decision, but also provide means that they can act on to obtain a desried outcome.

GAM Coach helps you generate diverse, optimal, and customizable counterfactual explantions for EBMs. Internally, GAM Coach formulates and solves a [mixed-integer linear programming](https://en.wikipedia.org/wiki/Integer_programming) problem.

In this section, we will learn how to generate counterfactual explantions using GAM Coach.

In [1]:
ebm.term_scores_[0]

NameError: name 'ebm' is not defined

In [7]:
# Find an interesting data point
# We can focus on test cases where our model rejcets the application (y_hat = 0)
reject_index = y_pred == 0
x_reject = x_test[reject_index, :]
y_pred_reject = y_pred[reject_index]

reject_df = pd.DataFrame(np.hstack((x_reject, y_pred_reject.reshape(-1, 1))))
reject_df.columns = feature_names + ['prediction']
reject_df.head()

,loan_amnt,term,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies,fico_score,prediction
0,10000.0,36 months,< 1 year,OWN,4.531478917042255,Source Verified,credit_card,31.66,2008.0,7.0,0,3.760422483423212,51.0,12.0,Individual,0,0,677.0,0
1,10000.0,60 months,1 year,RENT,4.643452676486188,Not Verified,debt_consolidation,4.34,2014.0,7.0,0,3.671358003443492,26.7,7.0,Individual,0,0,662.0,0
2,11000.0,36 months,8 years,MORTGAGE,4.556302500767287,Source Verified,major_purchase,13.4,2011.0,12.0,1,3.8899736384039962,44.9,15.0,Individual,0,1,677.0,0
3,15400.0,60 months,3 years,RENT,4.763427993562937,Verified,debt_consolidation,27.18,2005.0,9.0,0,3.940217555599735,68.1,28.0,Individual,0,0,682.0,0
4,10725.0,60 months,4 years,RENT,4.698970004336019,Source Verified,house,27.63,2004.0,9.0,0,3.688864568054792,20.5,19.0,Individual,0,0,772.0,0


In [8]:
# Find a random sample
rs = np.random.RandomState(SEED)
target_index = rs.choice(range(reject_df.shape[0]))
cur_example = x_reject[target_index, :]

print(cur_example)

['10375.0' '60 months' '10+ years' 'RENT' '4.672097857935717' 'Verified'
 'debt_consolidation' '25.07' '1995.0' '7.0' '0' '4.124243308946649'
 '87.6' '23.0' 'Individual' '0' '0' '702.0']


In [9]:
# Generate counterfactual (CF) explanations

# First create a GAM Coach object
# It requires to provide the training data, so it can generate better CFs based
# on the data distribution
my_coach = coach.GAMCoach(ebm, x_train)

In [10]:
# To generate good CF explantions, we need to consider what features are mutable
# and they possible values

cfs = my_coach.generate_cfs(
    cur_example,
    total_cfs=10,
    # List of features that the CFs can change
    features_to_vary=['loan_amnt', 'term', 'emp_length', 'home_ownership',
                      'annual_inc', 'purpose', 'dti', 'open_acc', 'revol_bal',
                      'revol_util', 'total_acc', 'application_type', 'mort_acc',
                      'fico_score'],
    # Some continuous features need to have integer values in practice
    continuous_integer_features=['open_acc', 'total_acc', 'mort_acc', 'fico_score'],
)

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


In [11]:
# View counterfactual examples as a dataframe
cfs.to_df()

,loan_amnt,term,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,application_type,mort_acc,pub_rec_bankruptcies,fico_score,new_prediction
0,10362.4999,36 months,10+ years,RENT,4.672097857935717,Verified,debt_consolidation,25.07,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,705.0,1
1,7912.4999,60 months,10+ years,RENT,4.70741684156812,Verified,debt_consolidation,22.8299,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,715.0,1
2,1449.9999,60 months,10+ years,RENT,4.7159866393584,Verified,debt_consolidation,25.07,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,710.0,1
3,10375.0,60 months,10+ years,RENT,4.641966414881926,Verified,debt_consolidation,25.07,1995.0,7.0,0,3.577334315474175,87.6,23.0,Individual,0,0,702.0,1
4,7512.4999,60 months,10+ years,RENT,4.876504107510444,Verified,debt_consolidation,21.9549,1995.0,7.0,0,4.124243308946649,91.65,23.0,Individual,0,0,702.0,1
5,10375.0,60 months,10+ years,RENT,4.709084911374745,Verified,wedding,25.07,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,702.0,1
6,7462.4999,60 months,10+ years,NONE,4.672097857935717,Verified,debt_consolidation,25.07,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,702.0,1
7,10375.0,60 months,10+ years,RENT,4.672097857935717,Verified,debt_consolidation,19.4449,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,740.0,1
8,7262.4999,60 months,10+ years,OTHER,4.672097857935717,Verified,debt_consolidation,25.07,1995.0,7.0,0,4.124243308946649,87.6,23.0,Individual,0,0,702.0,1
9,10375.0,60 months,10+ years,RENT,4.699665844033426,Verified,debt_consolidation,21.8349,1995.0,7.0,0,4.124243308946649,86.7499,23.0,Individual,0,0,745.0,1


In [12]:
# View counterfactual examples as "strategies" in text format
cfs.show()

## Strategy 1 ##
Change <loan_amnt> from 10375.0 to 10362.4999 [10225.0, 10362.5)
	* score gain: 0.0193
	* distance cost: 0.0021
Change <term> from "60 months" to "36 months" 
	* score gain: 1.0256
	* distance cost: 0.5360
Change <fico_score> from 702.0 to 705.0 [704.5, 709.5)
	* score gain: 0.0509
	* distance cost: 0.1500

## Strategy 2 ##
Change <loan_amnt> from 10375.0 to 7912.4999 [7775.0, 7912.5)
	* score gain: 0.1703
	* distance cost: 0.4104
Change <annual_inc> from 4.672097857935717 to 4.70741684156812 [4.70741684156812, 4.709084911374745)
	* score gain: 0.1537
	* distance cost: 0.2300
Change <dti> from 25.07 to 22.8299 [22.735, 22.83)
	* score gain: 0.3663
	* distance cost: 0.3852
Change <fico_score> from 702.0 to 715.0 [714.5, 719.5)
	* score gain: 0.4019
	* distance cost: 0.6500
Trigger interaction term: <dti x fico_score>
	* score gain: 0.0000
	* distance cost: 0.0000

## Strategy 3 ##
Change <loan_amnt> from 10375.0 to 1449.9999 [1250.0, 1450.0)
	* score gain: 0.6776
	* dis

## 4. Customize Counterfactual Generations

In practice, generating good counterfactul (CF) explanations requires model builders to contextualize the use cases of these explantions.
For example, to generate **actionable** CF explanations, one needs to consider what feature is possible to change and easy to change.

By default, GAM Coach varries the distance measurement of each feature by taking account of its deviation in the trianing data.
In other words, features that have higher deviations are considereed "easier to change," and therefore have relatively smaller distance penaties.
To do that, GAM Coach normalizes continuous features's [L1 distance](https://en.wikipedia.org/wiki/Taxicab_geometry) with [median absolute deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) and uses [complement probability](https://en.wikipedia.org/wiki/Complementary_event) to measure the distance between different levels within each categorical feature. Finally, GAM Coach recalibrates the distance scores of categorical features so that they have the same average score as continuous features.

However, these automatically generated distance functions are not sufficient to model the complexity of real-life problems. For example, based on the training data alone, it is hard to tell if increasing `income` by $1k is easier or harder than increasing `FICO score` by 5 points for individual end users. Therefore, GAM Coach provides an easy-to-use API to enable model builders to customize counterfactual generations. For example, we can easily define the distance scores by providing `cont_mads` and `cat_distnaces` when instanciating a `GAMCoach` object. Moreover, we can impose mroe constraints to the the CF example generation. You can learn more from reading the API documentations by calling `?my_coach` and `?my_coach.generate_cfs`.

In [13]:
#?my_coach

In [14]:
#?my_coach.generate_cfs

There is a trade-off between the *number of features* that a CF can change and the *distance* between the generated CF and the original data point.
However, in practice we tend to prefer CF explantions that change fewer features.
Here we demonstrate how to control the sparsity of CF examples. We can easily impose this constraint by specifying the `max_num_features_to_vary` argument in `my_coach.generate_cfs`.

In [15]:
# Here we allow GAM Coach to explore 14 features, but only change at most 2 features
# for each CF example.
# GAM Coach will find the best CF exampels that meet this constraint.

cfs = my_coach.generate_cfs(
    cur_example,
    total_cfs=5,
    # List of features that the CFs can change
    features_to_vary=['loan_amnt', 'term', 'emp_length', 'home_ownership',
                      'annual_inc', 'purpose', 'dti', 'open_acc', 'revol_bal',
                      'revol_util', 'total_acc', 'application_type', 'mort_acc',
                      'fico_score'],
    # Some continuous features need to have integer values in practice
    continuous_integer_features=['open_acc', 'total_acc', 'mort_acc', 'fico_score'],
    # Only change at most 2 features for each CF
    max_num_features_to_vary=2,
)

100%|██████████| 5/5 [00:05<00:00,  1.11s/it]


In [16]:
# View counterfactual examples as "strategies" in text format
cfs.show()

## Strategy 1 ##
Change <term> from "60 months" to "36 months" 
	* score gain: 1.0256
	* distance cost: 0.5360
Change <annual_inc> from 4.672097857935717 to 4.699665844033426 [4.699665844033426, 4.702637244924677)
	* score gain: 0.0575
	* distance cost: 0.1795

## Strategy 2 ##
Change <annual_inc> from 4.672097857935717 to 4.641966414881926 [4.634476101197249, 4.642066414881926)
	* score gain: 0.1296
	* distance cost: 0.1955
Change <revol_bal> from 4.124243308946649 to 3.577334315474175 [3.568201581311275, 3.577434315474175)
	* score gain: 0.8746
	* distance cost: 2.1671
Trigger interaction term: <annual_inc x revol_bal>
	* score gain: 0.0851
	* distance cost: 0.0000

## Strategy 3 ##
Change <loan_amnt> from 10375.0 to 1449.9999 [1250.0, 1450.0)
	* score gain: 0.6776
	* distance cost: 1.4875
Change <dti> from 25.07 to 19.4449 [19.415, 19.445)
	* score gain: 0.4569
	* distance cost: 0.9673

## Strategy 4 ##
Change <annual_inc> from 4.672097857935717 to 4.70741684156812 [4.70741684156812

We can see that GAM Coach suggests the different strategies. Even though it is doubtful if the loan applicant can easily change the loan purpose from "debt_consolidation" to "wedding", the new strategy looks easier to understand than the old stragety (need to change `loan_amnt`, `annual_inc`, and `dti` to certain ranges).

## 5. Conclusion

In this notebook, we learn about how to use GAM Coach to generate diverse counterfactual explantions for an EBM binary classifier and customize the generation process. To learn more about GAM Coach, you can check out [our paper](https://arxiv.org/abs/2302.14165) or raise an issue on [GitHub](https://github.com/xiaohk/gam-coach/issues). Happy coaching!